In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from denmarf import DensityEstimate
from denmarf.utils import plot_corner

from sklearn.neighbors import KernelDensity
from sklearn.mixture import GaussianMixture


In [2]:
data = pd.read_csv("sample_1C.csv", skiprows=5)

In [3]:
data.head()

,distance,plx,s_plx,q_obs,s_q,u_obs,s_u,plx_true,q_true,u_true,distance_true
0,188.985028,5.291424,0.02,0.001276,0.001003,0.000170,0.001003,5.28,0.000000,0.000000,189.393939
1,189.773494,5.269440,0.02,0.000759,0.001004,-0.001881,0.001004,5.28,0.000000,0.000000,189.393939
2,187.664165,5.328668,0.02,0.000520,0.001009,-0.000452,0.001009,5.28,0.000000,0.000000,189.393939
3,190.720114,5.243285,0.02,0.000493,0.001013,0.001084,0.001013,5.24,0.000000,0.000000,190.839695
4,524.770654,1.905594,0.02,0.005737,0.001028,0.001726,0.001028,1.91,0.005158,0.001053,523.560209


In [4]:
plt.errorbar(data["distance"], data["q_obs"], data["s_q"], ls='', marker='.', label="q")
plt.errorbar(data["distance"], data["u_obs"], data["s_u"], ls='', marker='.', label="u")
plt.legend()
plt.show()


# Not considering errors for now

In [ ]:
nfde_q = DensityEstimate().fit(data[["distance","q_obs"]])
nfde_q.save("1C_nfde_q.pkl")

current average log likelihood: -7.608: 100%|██████████| 1000/1000 [07:00<00:00,  2.38it/s]   

best average log likelihood: -4.291


In [ ]:
nfde_u = DensityEstimate().fit(data[["distance","u_obs"]])
nfde_u.save("1C_nfde_u.pkl")

current average log likelihood: -5.317: 100%|██████████| 1000/1000 [08:16<00:00,  2.02it/s]

best average log likelihood: -5.066


In [5]:
import torch
nfde_q = torch.load("1C_nfde_q.pkl", weights_only=False)
nfde_u = torch.load("1C_nfde_u.pkl", weights_only=False)

In [6]:
N = 100
x = np.linspace(0,5000, N)
y = np.linspace(-0.01, 0.02, N)
XX, YY = np.meshgrid(x,y)
XY = np.column_stack((XX.ravel(), YY.ravel()))

Zq = nfde_q.score_samples(XY)
Zq = Zq.reshape(N,N)
Zu = nfde_u.score_samples(XY)
Zu = Zu.reshape(N,N)

In [7]:
fig, ax = plt.subplots(2,2, figsize=[12,6], height_ratios=[20,1])
cf1 = ax[0,0].contourf(XX, YY, Zq, levels=50, cmap="Blues_r")
ax[0,0].scatter(data["distance"], data["q_obs"], s=5)
cf2 = ax[0,1].contourf(XX, YY, Zu, levels=50, cmap="Blues_r")
ax[0,1].scatter(data["distance"], data["u_obs"], s=5)

plt.colorbar(cf1, cax=ax[1,0], orientation="horizontal")
plt.colorbar(cf2, cax=ax[1,1], orientation="horizontal")
plt.show()


# KDE

In [14]:
kde_q = KernelDensity(kernel='gaussian', bandwidth='scott').fit(data[["distance","q_obs"]])
kde_u = KernelDensity(kernel='gaussian', bandwidth='scott').fit(data[["distance","u_obs"]])


In [15]:
Zkde_q = kde_q.score_samples(XY)
Zkde_q = Zkde_q.reshape(N,N)
Zkde_u = kde_u.score_samples(XY)
Zkde_u = Zkde_u.reshape(N,N)


/home/kopty/.miniconda3/envs/ML/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but KernelDensity was fitted with feature names
  warnings.warn(
/home/kopty/.miniconda3/envs/ML/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but KernelDensity was fitted with feature names
  warnings.warn(


In [16]:
fig, ax = plt.subplots(2,2, figsize=[12,6], height_ratios=[20,1])
cf1 = ax[0,0].contourf(XX, YY, Zkde_q, levels=50, cmap="Blues_r")
ax[0,0].scatter(data["distance"], data["q_obs"], s=5)
cf2 = ax[0,1].contourf(XX, YY, Zkde_u, levels=50, cmap="Blues_r")
ax[0,1].scatter(data["distance"], data["u_obs"], s=5)

plt.colorbar(cf1, cax=ax[1,0], orientation="horizontal")
plt.colorbar(cf2, cax=ax[1,1], orientation="horizontal")
plt.show()
